In [3]:
import json

datapath='data/'
secret_file = '../secrets/gcloud.json'

with open(secret_file) as json_file:
    secrets = json.load(json_file)
    api_key=secrets['api_key']

In [9]:
import googlemaps
import math

def get_city_coordinates(api_key, city):
    gmaps = googlemaps.Client(key=api_key)
    geocode_result = gmaps.geocode(city)

    if len(geocode_result) > 0:
        location = geocode_result[0]['geometry']['location']
        return location['lat'], location['lng']
    else:
        return None

def calculate_destination_coordinates(api_key, city1, distance1, city2, distance2):
    earth_radius_km = 6371.0

    # Get the coordinates of the two cities
    city1_lat, city1_lng = get_city_coordinates(api_key, city1)
    city2_lat, city2_lng = get_city_coordinates(api_key, city2)

    if city1_lat is None or city1_lng is None or city2_lat is None or city2_lng is None:
        return None

    # Convert distances to radians
    distance1_rad = distance1 / earth_radius_km
    distance2_rad = distance2 / earth_radius_km

    # Calculate the coordinates of the destination location
    delta_lng = math.radians(city2_lng - city1_lng)
    A = math.sin(distance1_rad) ** 2 + math.cos(math.radians(city1_lat)) * math.cos(math.radians(city2_lat)) * math.sin(delta_lng / 2) ** 2
    C = 2 * math.atan2(math.sqrt(A), math.sqrt(1 - A))

    destination_lat = math.degrees(math.asin(math.sin(math.radians(city1_lat)) * math.cos(distance1_rad) + math.cos(math.radians(city1_lat)) * math.sin(distance1_rad) * math.cos(C)))
    destination_lng = city1_lng + math.degrees(math.atan2(math.sin(delta_lng) * math.sin(distance1_rad) * math.cos(math.radians(city1_lat)), math.cos(distance1_rad) - math.sin(math.radians(city1_lat)) * math.sin(math.radians(destination_lat))))

    return destination_lat, destination_lng

if __name__ == "__main__":
    api_key = api_key
    city1 = "Carrare, Italy"
    distance1 = 259  # distance in km from CityA
    #city2 = "Pardess Hana, Israel"
    #distance2 = 2739  # distance in km from CityB
    city2 = "Murcia, Spain"
    distance2 = 927  # distance in km from CityB
    #city2 = "Marblehead, Massachusetts"
    #distance2 = 6079  # distance in km from CityB
    

    destination_lat, destination_lng = calculate_destination_coordinates(api_key, city1, distance1, city2, distance2)
    print("Destination coordinates (latitude, longitude):", destination_lat, destination_lng)


Destination coordinates (latitude, longitude): 46.37427295294397 9.44048507435125


In [12]:
from geopy.distance import distance
import math

def calculate_destination_coordinates(lat1, lon1, distance_km1, lat2, lon2, distance_km2):
    # Earth's radius in kilometers
    earth_radius_km = 6371.0

    # Convert distances to radians
    distance_rad1 = distance_km1 / earth_radius_km
    distance_rad2 = distance_km2 / earth_radius_km

    # Convert latitudes and longitudes to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate the two possible lat/lon pairs using the haversine formula
    lat_dest1 = math.asin(math.sin(lat1_rad) * math.cos(distance_rad1) + math.cos(lat1_rad) * math.sin(distance_rad1) * math.cos(0))
    lon_dest1 = lon1_rad + math.atan2(math.sin(0) * math.sin(distance_rad1) * math.cos(lat1_rad),
                                       math.cos(distance_rad1) - math.sin(lat1_rad) * math.sin(lat_dest1))

    lat_dest2 = math.asin(math.sin(lat2_rad) * math.cos(distance_rad2) + math.cos(lat2_rad) * math.sin(distance_rad2) * math.cos(math.pi))
    lon_dest2 = lon2_rad + math.atan2(math.sin(math.pi) * math.sin(distance_rad2) * math.cos(lat2_rad),
                                       math.cos(distance_rad2) - math.sin(lat2_rad) * math.sin(lat_dest2))

    # Convert back to degrees
    lat_dest1 = math.degrees(lat_dest1)
    lon_dest1 = math.degrees(lon_dest1)
    lat_dest2 = math.degrees(lat_dest2)
    lon_dest2 = math.degrees(lon_dest2)

    return (lat_dest1, lon_dest1), (lat_dest2, lon_dest2)

# Example usage:
city1_lat, city1_lon = 40.7128, -74.0060  # Latitude and longitude of City 1 (e.g., New York City)
city2_lat, city2_lon = 34.0522, -118.2437  # Latitude and longitude of City 2 (e.g., Los Angeles)
distance1 = 100  # km
distance2 = 200  # km

lat_lon_pair_1, lat_lon_pair_2 = calculate_destination_coordinates(city1_lat, city1_lon, distance1, city2_lat, city2_lon, distance2)
print(f"Lat/Lon Pair 1: {lat_lon_pair_1}")
print(f"Lat/Lon Pair 2: {lat_lon_pair_2}")


Lat/Lon Pair 1: (41.61212160591874, -74.006)
Lat/Lon Pair 2: (32.25355678816254, -118.2437)


In [15]:
from geopy.geocoders import GoogleV3
import math

def find_coordinates_at_distances(city1_name, city2_name, distance1_km, distance2_km):
    # Initialize Google Maps API client
    geolocator = GoogleV3(api_key=api_key)  # Replace 'YOUR_GOOGLE_MAPS_API_KEY' with your API key

    # Find the lat/lon of the two cities
    location1 = geolocator.geocode(city1_name)
    location2 = geolocator.geocode(city2_name)

    # Check if locations are found
    if not location1:
        raise ValueError(f"Location for {city1_name} not found.")
    if not location2:
        raise ValueError(f"Location for {city2_name} not found.")

    city1_lat, city1_lon = location1.latitude, location1.longitude
    city2_lat, city2_lon = location2.latitude, location2.longitude

    # Call the calculate_destination_coordinates function with the obtained coordinates
    lat_lon_pair_1, lat_lon_pair_2 = calculate_destination_coordinates(city1_lat, city1_lon, distance1_km, city2_lat, city2_lon, distance2_km)
    return lat_lon_pair_1, lat_lon_pair_2

# The calculate_destination_coordinates function as defined earlier

# Example usage:
city1_name = "Carrare, Italy"
city2_name = "Murcia, Spain"
distance1_km = 259  # km
distance2_km = 927  # km
city1_name = 'Pardess Hana, Israel'
distance1_km = 2739

lat_lon_pair_1, lat_lon_pair_2 = find_coordinates_at_distances(city1_name, city2_name, distance1_km, distance2_km)
print(f"Lat/Lon Pair 1: {lat_lon_pair_1}")
print(f"Lat/Lon Pair 2: {lat_lon_pair_2}")


Lat/Lon Pair 1: (57.10522158611401, 34.9742001)
Lat/Lon Pair 2: (29.655528613133377, -1.130654399999999)
